In [1]:
import numpy as np
from math import pi, tau

from datetime import datetime, timezone
from tzwhere import tzwhere
import pytz
import suncalc

from colorsys import hls_to_rgb
from PIL import Image

In [2]:
# array of UTC timestamps as naive datetime objects
def base_date_arr(year: int):
    start_time = np.datetime64(str(year))
    end_time = np.datetime64(str(year + 1))

    arr_dt_1d = np.arange(start_time, end_time, dtype='datetime64[m]').astype(datetime)  # 1D array

    return arr_dt_1d

In [3]:
# array of naive timestamps to array of time zone aware timestamps
def to_utc(arr, lon, lat, use_dst: bool):
    tz_str = tzwhere.tzwhere().tzNameAt(lon, lat)
    tz = pytz.timezone(tz_str)

    def one_dt_utc(dt):
        return (dt - offset).replace(tzinfo=timezone.utc)

    def one_dt_localized(dt):
        return tz.localize(dt).astimezone(pytz.utc)

    if use_dst:
        return np.vectorize(one_dt_localized)(arr)
    else:
        # offset = tz.utcoffset(arr[0, 0], is_dst=False)
        offset = tz.utcoffset(arr[0], is_dst=False)
        return np.vectorize(one_dt_utc)(arr)

In [4]:
# azimuth, altitude to color
def get_color(azimuth, altitude, sunrise_jump=0.2, hue_shift=0.0):
    # ranges from 0 (no jump) to 1 (day is all white, night all black)
    assert 0 <= sunrise_jump <= 1, "sunrise_jump must be between 0 and 1 inclusive"

    # can be any float, but values outside the range [0, 1) are redundant
    # shifts all hues in the r -> g -> b -> r direction
    assert isinstance(hue_shift, float), "hue_shift must be a float"

    # yes, this could be simplified, and no, don't try to do it please.
    altitude_scaled = (altitude / pi) * 2  # range [-1, 1]
    altitude_scaled *= 1 - sunrise_jump
    # altitude_scaled += sunrise_jump * (1 if altitude >= 0 else -1)  # range [-1, 1]
    idx_alt_pos = altitude_scaled >= 0
    idx_alt_neg = altitude_scaled < 0
    #altitude_scaled[idx_alt_pos] += sunrise_jump * 1
    #altitude_scaled[idx_alt_neg] += sunrise_jump * -1
    altitude_scaled[idx_alt_pos] += sunrise_jump
    altitude_scaled[idx_alt_neg] -= sunrise_jump
    lightness = altitude_scaled / 2 + 0.5  # range [0, 1]

    hue = ((azimuth / tau) + 0.5 + hue_shift) % 1  # range [0, 1]
    
    saturation = np.ones(hue.shape)
    
    r, g, b = np.vectorize(hls_to_rgb)(hue, lightness, saturation)
    
    r = np.round(255 * r)
    g = np.round(255 * g)
    b = np.round(255 * b)
    
    return r, g, b

In [5]:
def stack_rgb(r, g, b):
    new = np.empty((1440,365,3), dtype=np.uint8)
    new[:,:,0] = r.reshape((-1,1440)).T
    new[:,:,1] = g.reshape((-1,1440)).T
    new[:,:,2] = b.reshape((-1,1440)).T
    return new
    #return np.vstack((r, g, b)).reshape((-1,), order='F').reshape((1440,365,3)).astype(np.uint8)

In [6]:
# generate PNG using pixel data
def gen_png(rgb_arr, width, height, file_name):
    if not '.png' in file_name: file_name = file_name + '.png'
    Image.fromarray(rgb_arr, mode="RGB") \
        .resize((rgb_arr.shape[1], height), Image.BOX) \
        .resize((width, height), Image.NEAREST) \
        .save(file_name)

In [7]:
year = 2021

img_title = 'Melbourne_new_6'
lon = 144.9631
lat = -37.8136

width = 1920
height = 1080

print('Building array of datetime objects...')
bef = datetime.now()
arr_dt = base_date_arr(year)  # naive timestamps as datetime.datetime
af = datetime.now() - bef
print(f'Took {af}')

print('Converting dates to UTC...')
bef = datetime.now()
arr_utc = to_utc(arr_dt, lat, lon, use_dst=True)
af = datetime.now() - bef
print(f'Took {af}')

print('Calculating sun location...')
bef = datetime.now()
sc = suncalc.get_position(arr_utc, lon, lat)
af = datetime.now() - bef
print(f'Took {af}')

# azi = sc['azimuth'].reshape((1440,-1))
# alt = sc['altitude'].reshape((1440,-1))

print('Calculating colors...')
bef = datetime.now()
r, g, b = get_color(sc['azimuth'], sc['altitude'], sunrise_jump=0.3, hue_shift=0.0)
af = bef = datetime.now() - bef
print(f'Took {af}')

print('Interleaving pixels...')
bef = datetime.now()
pixels = stack_rgb(r, g, b)
af = datetime.now() - bef
print(f'Took {af}')

print('Generating image...')
bef = datetime.now()
gen_png(pixels, width, height, img_title)
af = datetime.now() - bef
print(f'Took {af}')

Building array of datetime objects...
Took 0:00:00.038828
Converting dates to UTC...


/home/anoush/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Took 0:00:12.504849
Calculating sun location...
Took 0:00:00.231002
Calculating colors...
Took 0:00:00.481923
Interleaving pixels...
Took 0:00:00.003360
Generating image...
Took 0:00:00.208940
